In [471]:
import numpy as np
from graphviz import Digraph
import os, shutil
import glob
import imageio




In [586]:
class Edge:
    def __init__(self, _from, _vowel, _to):
        self._from = _from
        self._vowel = _vowel
        self._to= _to
    
def initialFinalStates(line1):
    nodesInitial = []
    nodesFinals = []
    #separar pelo ;
    aux=line1.split(';')
    #separar os iniciais da 1ª parte
    for initial in aux[0].split(' '):
        if(initial!='' ):
            nodesInitial.append(initial)
    #separar os finais da 2ª parte          
    for final in aux[1].split(' '):
        if(final!= ''):
            nodesFinals.append(final)
    return nodesInitial, nodesFinals

def getNodes(lines):
    nodes=[]
    #pegar cada linha l retirando a 1ª e ultima
    for l in lines[1:-1]:
        #separar "s0 a > s0"
        #cada 1ª informação equivale a um nó
        node = l.split(' ')[0]
        if node not in nodes:
            nodes.append(node)
    return nodes

def getEdges(lines):
    edges=[]
    #pegar cada linha l retirando a 1ª e ultima
    for l in lines[1:-1]:
        #separar "s0 a > s0" -> s0 se 'a' vai para s0
        edge = l.split(' ')
        if edge!='':
            #"s0 a > s0"
            _from = edge[0]#node de saida
            _vowel = edge[1]# vogal
            _road = edge[-2]#caminho
            _to = edge[-1]#node de entrada
           
            if(_road==">"):
                newEdge = Edge(_from,_vowel,_to)
            else:
                _from,_to = _to,_from #swap
                newEdge = Edge(_from,_vowel,_to)
                
            edges.append(newEdge)
    return edges

def getWord(lastLine):
    wrdLine = lastLine.replace(' ','').split(':')
    return wrdLine[-1] #pegar ultimo item 


def RenderIMGs(nodes,edges,initials, finals,wrd):

    count=1
    fileN = 'img/step0'
    A = Digraph('',format='png', filename=fileN)
    
    Str=''
    currentNode = initials[0]
    currentEdge = ''
    wrd = '|'+wrd
    first = True
    #desenhar cada frame:
    for v in wrd:
        A.clear()
        currEdgeDrawing = False
        #draw nodes
        for n in nodes:
            if n in initials:
                A.attr(rankdir='LR')
                A.attr('node', shape='point')
                A.node('',color='white')
                
                if n not in finals:
                    A.attr('node', shape='octagon')
                    A.node(n,color='blue')
                else:
                    A.attr('node', shape='doubleoctagon')
                    A.node(n,color='blue2')
                    
                A.attr('edge')
                if Str=='':
                    A.edge('',n,label='i',color='red')
                    currEdgeDrawing=True
                else:
                    A.edge('',n,label='i',color='black')
            elif n in finals:
                A.attr('node', shape='doublecircle')
                A.node(n,color='blue2')
            else:
                A.attr('node', shape='circle')
                A.node(n,color='black')
            #style finals
            
            if first:
                first=False
                if n in finals:
                    A.attr('node', shape='doublecircle')
                    A.node(n,style='filled', fillcolor='green3')
                elif n in initials:
                    A.node(n,style='filled', fillcolor='yellow')
                else:
                    A.node(n,style='filled', fillcolor='red')
                    
        #draw Edge        
        for e in edges:
            if e._vowel == v and e._from == currentNode and not currEdgeDrawing :
                A.edge(e._from , e._to, label=e._vowel, color='red')
                currEdgeDrawing=True
                currentNode = e._to
                if currentNode in finals and currentNode in nodes:
                    A.node(currentNode,style='filled', color='green')
                else:
                    A.node(currentNode,style='filled', fillcolor='red3')
               
            else:
                A.edge(e._from , e._to, label=e._vowel, color='black')
                
            
        
        #alterar o current node e edge
        
        
        #draw frame
        _label = Str +v.upper() +'|'+ wrd[count:]
        Str = Str + v
        A.attr(label=_label)
        A.attr(fontsize='20')

        #A.view()
        aux = "img/step"+str('%03d'%count)
        
        A.render(aux, view=False) 
        count=count+1
    #renderizar mais uma imagem
    aux = "img/step"+str('%03d'%count)
    A.render(aux, view=False)
    
    
def clearFolder(path):
    folder = 'img/*'
    files = glob.glob(folder)
    for f in files:
        os.remove(f)
    
        
def generateGif(path):
    png_dir ='img'
    images = []
    for file_name in sorted(os.listdir(png_dir)):
        if file_name.endswith('.png'):
            file_path = os.path.join(png_dir, file_name)
            images.append(imageio.imread(file_path))
    imageio.mimsave('movie.gif', images,fps=1)

def interpreterAF(path):
    clearFolder(path)
    #lendo o arquivo
    with open(path, 'r') as file:
        data = file.read()
    data = data.split("\n")
    
    #retirando informações
    nodes= getNodes(data)
    edges= getEdges(data)
    #separar os nodes iniciais e finais
    initials, finals = initialFinalStates(data[0])
    #pegando a palavra
    wrd = getWord(data[-1])
    
    #agora temos que criar as imagens
     #uma para cada coneceção/edge
    print('init: ',initials)
    print('final: ',finals)
        
    RenderIMGs(nodes,edges,initials, finals,wrd)
    generateGif(path)

    


In [587]:
interpreterAF('file.txt')

init:  ['s0']
final:  ['s0']


In [540]:
images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(path+'/AF.gif', images)
    #For longer movies, use the streaming approach:
    
    #import imageio
    with imageio.get_writer('AF.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
    
    #https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python

IndentationError: unexpected indent (<ipython-input-540-4ab323149b48>, line 2)

In [503]:
count =4
aux = "img/step"+str('%05d'%count)

print(aux)

img/step00004


In [526]:
'/'.join(['aaa','bbb'])

'aaa/bbb'